In [3]:
def xADD(XP, ZP, XQ, ZQ, X_diff, p):
    F = GF(p)
    A = F(XP + ZP)
    B = F(XP - ZP)
    C = F(XQ + ZQ)
    D = F(XQ - ZQ)
    DA = D * A
    CB = C * B
    X_out = (DA + CB)^2
    Z_out = X_diff * (DA - CB)^2
    return X_out, Z_out


#xP=P[0]

def kummer_poly(xP, xQ, A):
    F = xP.parent()
    R.<X> = PolynomialRing(F)
    num = (xP * xQ - 1)^2
    den = (xP - xQ)^2
    trace = -2 * ((xP * xQ + 1)*(xP + xQ) + 2*A*xP*xQ) / den
    product = num / den
    return X^2 + trace*X + product, trace, product # retourne 3 objets

def kummer_poly_proj(XP, ZP, XQ, ZQ, A):
    F = XP.parent()
    R.<X> = PolynomialRing(F)
    # On calcule les invariants homogènes : trace et produit
    num_trace = -2 * ( (XP*XQ + ZP*ZQ)*(XP*ZQ + XQ*ZP) + 2*A*XP*XQ*ZP*ZQ )
    den_trace = (XP*ZQ - XQ*ZP)^2

    num_prod = (XP*XQ - ZP*ZQ)^2
    den_prod = (XP*ZQ - XQ*ZP)^2

    # Homogénéisation : on met tout sous le même dénominateur (commun = den_prod*den_trace)
    trace = num_trace 
    product = num_prod

    return den_trace * X^2 + trace * X + product, trace, product


#Données à avoir : P[0], (Q+T)[0], Q[0], (P+T)[0]
def addition_compatible(x1, x2, x3, x4, A):
    f, a1, b1 = kummer_poly(x1, x2, A)
    g, a2, b2 = kummer_poly(x3, x4, A)
    if a1 == a2:
        raise ZeroDivisionError("Les traces sont égales, le PGCD n’est pas de degré 1.")
    racine = -(b1 - b2) / (a1 - a2)  # X + racine = 0  ⇨ racine = -cst/coef & #car le pgcd de 2 polynômes X^2+a_1X+b_1 et X^2+a_2X+b_2 est X + (b_1-b_2)/(a_1-a_2)
    return racine


def addition_compatible_proj(X1, Z1, X2, Z2, X3, Z3, X4, Z4, A):
    F = X1.parent()
    R.<X> = PolynomialRing(F)

    f, a1, b1 = kummer_poly_proj(X1, Z1, X2, Z2, A)
    g, a2, b2 = kummer_poly_proj(X3, Z3, X4, Z4, A)

    if a1 == a2:
        raise ZeroDivisionError("PGCD indéterminé : traces égales.")

    num = -(b1 - b2)
    den = (a1 - a2)

    x_affine = num / den

    # Retour en projectif : x = X/Z ⇒ (X, Z) = (x_affine, 1)
    return x_affine * F(1), F(1)  # (X, Z)


p = 2^255 - 19
F = GF(p)
A = F(486662)

# exemple de points 
xP = F(9)
xQ = F(5)
xT = F(7)
xPplusT = F(12)
xQplusT = F(14)

xPplusQplusT = addition_compatible(xP, xQplusT, xQ, xPplusT, A)

xQminusT = ((xQ * xT - 1)^2) / ((xQ - xT)^2 * xQplusT)

xPplusQ = addition_compatible(xP, xQ, xPplusT, xQminusT, A)

In [4]:
xPplusQplusT

32923605311662302840749844416638765527765399322374623388614743548863489562491

In [5]:
xPplusQ

55932831376119066159002051292036098821001550643621559099532083783896093991274